In [36]:
import json
import numpy as np
import pandas as pd 
import seaborn as sns
from commands import COMMANDS
from matplotlib import pyplot as plt

sns.set_style('whitegrid')

In [56]:
data = pd.read_csv('../data/ad_music_proccessed.csv', index_col=0)
leads = pd.read_csv('../data/leads.csv', index_col=0)

In [57]:
ato_lead = leads.iloc[2]
ato_lead[ato_lead != 0]

cookie                               9018f5d85ecf8185385f99cab02d48f7
unique_user_ids     ['U782372', 'U621865', 'U401248', 'U148321', '...
num_users                                                          13
target_artist                                                   [nan]
target_user                                                     [nan]
change_password                                                    13
disable_mfa                                                        13
download_info                                                      13
homepage                                                           14
personal_details                                                   13
view_full_info                                                     13
Name: 2, dtype: object

In [39]:
ck_data = data[data.cookie.eq(ato_lead.cookie)]
ck_data = ck_data[ck_data.command.isin(COMMANDS.ato)]
user_command_count = ck_data.groupby('user_id')['command'].nunique()
qualified_user_ids = user_command_count[user_command_count.eq(2)].index
len(qualified_user_ids)

13

In [40]:
ck_data = data[data.cookie.eq(ato_lead.cookie)]
ck_data = ck_data[ck_data.command.isin(COMMANDS.info_steal)]
user_command_count = ck_data.groupby('user_id')['command'].nunique()
qualified_user_ids = user_command_count[user_command_count.eq(3)].index
len(qualified_user_ids)

13

The cookie '9018f5d85ecf8185385f99cab02d48f7' has 13 accounts linked to it.
All of them changed passwords and disabled MFA - so they are hijacked (ATO).
All of these accounts have viewed thier personal info AND downloaded it - So this is information theft as well.

Affected accounts:

In [41]:
qualified_user_ids

Index(['U061389', 'U099019', 'U148321', 'U243397', 'U286051', 'U401248',
       'U416122', 'U440291', 'U545064', 'U621865', 'U707396', 'U782372',
       'U847514'],
      dtype='object', name='user_id')

General ATO cases

In [42]:
out = qualified_user_ids.values.tolist()

In [43]:
with open("affected_users.json", "r") as f:
    out_file = json.load(f)

out_file['ato'] = out_file['ato'] + out
out_file['information_theft'] = out

with open("affected_users.json", "w") as f:
    json.dump(out_file, f, indent=4)
    
known_cases =  out_file['ato']

In [44]:
ato_data = data[data.command.isin(COMMANDS.ato)]
user_command_count = ato_data.groupby('user_id')['command'].nunique()
general_user_ids = user_command_count[user_command_count.eq(2)].index
general_user_ids = general_user_ids.difference(qualified_user_ids)
general_user_ids = general_user_ids.difference(known_cases)
len(general_user_ids) 

10

10 Account have changed password AND disabled MFA (other than the ones we already know)

In [45]:
ato_data = data[data.user_id.isin(general_user_ids)]
pd.crosstab(ato_data['user_id'],ato_data['command'])

command,A035765,A121720,A409478,A520311,A762816,A783696,AL143326,AL274308,AL283235,AL356613,...,confirmation,disable_mfa,homepage,personal_details,search?q=classical,search?q=hip-hop,search?q=jazz,search?q=pop,search?q=rock,submit_purchase
user_id,,,,,,,,,,,,,,,,,,,,,
U164867,0,0,0,0,1,0,0,0,1,0,...,1,1,2,1,1,0,0,1,0,1
U168598,0,0,0,0,0,0,0,1,0,0,...,1,1,5,1,0,0,0,0,0,1
U368556,1,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,1,1
U438102,0,0,1,0,0,0,0,0,0,0,...,1,1,4,1,0,1,0,0,0,1
U671439,0,0,0,0,0,0,0,0,0,1,...,1,1,6,1,0,0,0,1,0,1
U687501,0,0,0,0,0,0,0,0,0,0,...,1,1,9,1,1,0,0,0,0,1
U744743,0,0,0,1,0,0,1,0,0,0,...,1,1,6,1,1,0,0,0,1,1
U777022,0,0,0,0,0,0,0,0,0,0,...,1,1,12,1,0,0,1,1,0,1
U883093,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,0,1,0,0,0,1


In [46]:
ato_data.cookie.value_counts()

cookie
5c887169d0d17b3a65b86fb6ba9a47bc    233
Name: count, dtype: int64

This is definetly ATO - they are all on the same cookie.

In [75]:
theft_lead = leads[[col for col in leads.columns if col not in COMMANDS.strange]].iloc[5]
theft_lead[theft_lead != 0]

cookie                                      5c887169d0d17b3a65b86fb6ba9a47bc
unique_user_ids            ['U744743', 'U438102', 'U168598', 'U883093', '...
num_users                                                                 10
target_artist              ['A903598', nan, 'A972244', 'A520311', 'A61677...
target_user                                                            [nan]
add_to_cart                                                               73
cart                                                                      10
change_billing_address                                                    10
change_password                                                           10
change_shipping_address                                                   10
confirmation                                                              10
disable_mfa                                                               10
homepage                                                                  52

In [47]:
theft_data = ato_data[ato_data.command.isin(COMMANDS.physical_theft)]
user_command_count = theft_data.groupby('user_id')['command'].nunique()
qualified_user_ids = user_command_count[user_command_count.eq(2)].index
len(qualified_user_ids)

10

In [52]:
theft_data = data[data.user_id.isin(qualified_user_ids)]
theft_data = theft_data[theft_data.command.eq('add_to_cart')]
user_command_count = theft_data.groupby('user_id')['command'].count()
user_command_count

user_id
U164867    10
U168598     6
U368556     9
U438102     8
U671439     5
U687501     7
U744743     5
U777022     5
U883093    10
U994777     8
Name: command, dtype: int64

In [53]:
user_command_count.sum()

np.int64(73)

All 10 accounts were used to submit purchase and changed shipping address. This is fraud. A total of 73 items were stolen.

In [49]:
strange_cmds = ato_data[ato_data.command.isin(COMMANDS.strange)]
user_command_count = strange_cmds.groupby('user_id')['command'].nunique()
qualified_user_ids = user_command_count[user_command_count > (0)].index
len(qualified_user_ids)

8

Also, 8 of these accounts have used unknown commnads. (Maybe cupon codes?)

In [50]:
out = theft_data.user_id.unique().tolist()

In [51]:
with open("affected_users.json", "r") as f:
    out_file = json.load(f)

out_file['ato'] = out_file['ato'] + out
out_file['theft'] = out

with open("affected_users.json", "w") as f:
    json.dump(out_file, f, indent=4)